# Search Algorithms

In this project, the aim is to solve a predefined problem using various search algorithms and compare their qualities. BFS, IDS and A* are the algorithms used for this problem.

## Problem Description

TBA

## Modeling

At first, the problem should be modeled as a graph so that the search algorithms can be applied to it in future steps. Each node representes one ditinct state of the map. Bellow, the components of the model is explained:

 - <b>Initial State:</b> The initial state of the map is given as input in this problem, and the player always starts at bottom left corner of the map.
 
 
 - <b>Actions:</b> At each step,one of the players can move either up, down, left or right. However, sometimes the players may be prevented from taking certain actions due to walls and obstacles.
 
 
 - <b>Transition Model:</b> TBA
 
 
 - <b>Goal State:</b> The goal state is when all of the players have reached the top right corner of the map and collected all potions in the map.
 
 
 - <b>Path Cost:</b> TBA
 

### Model Implementation

In [1]:
PLAYER='P'
POTION='M'
PILL='D'
OBSTACLE='O'

 - <b>Initiat State:</b>

In [2]:
def read_initial_state(file_name):
    file = open(file_name)
    row,col=file.readline().split()
    potion_count,pill_count=file.readline().split()
    initial_map={"row": int(row) , "col": int(col) , "potion_count": int(potion_count), "ploc":[]}
    for _ in range(int(potion_count)):
        x,y=file.readline().split()
        initial_map[(int(x),int(y))]=POTION
    for _ in range(int(pill_count)):
        x,y=file.readline().split()
        initial_map[(int(x),int(y))]=PILL
    obstacle_count=file.readline()
    for _ in range(int(obstacle_count)):
        x,y=file.readline().split()
        initial_map[(int(x),int(y))]=OBSTACLE
    initial_map["ploc"].append((0,0))
    return initial_map

 - <b>Actions:</b>

In [12]:
UP=(1,0)
DOWN=(-1,0)
RIGHT=(0,1)
LEFT=(0,-1)
ACTIONS=[UP,DOWN,LEFT,RIGHT]

 - <b>Transition Model:</b>

In [33]:
import copy as cp
def not_in_range(loc,loc_range):
    return loc[0]>=loc_range[0] or loc[1]>=loc_range[1] or loc[0]<0 or loc[1]<0
    

def move(current_state,player_number,direction):
    current_state=cp.deepcopy(current_state)
    current_location=current_state["ploc"][player_number]
    future_location=(current_location[0]+direction[0],current_location[1]+direction[1])
    if(future_location in current_state):
        if(current_state[future_location]==POTION):
            current_state["potion_count"]-=1;
        elif(current_state[future_location]==PILL):
            current_state["ploc"].append((0,0))
        elif(current_state[future_location]==OBSTACLE):
            return 0
    elif(not_in_range(future_location,(current_state["row"],current_state["col"]))):
        return 0
    current_location=current_state["ploc"][player_number]=future_location
    if(current_location in current_state):
        del current_state[current_location]
    return current_state

In [21]:
def is_goal_state(state):
    for loc in state["ploc"]:
        if(loc!=(state["row"]-1,state["col"]-1)):
            return False
    if(state["potion_count"]==0):
        return True
    else:
        return False

In [6]:
d=c.copy()
del d["row"]
print(c==d)
print(c,d)

False
{'row': 4, 'col': 4, 'potion_count': 2, 'ploc': [(0, 0)], (0, 2): 'M', (2, 0): 'M', (0, 1): 'D', (1, 2): 'O'} {'col': 4, 'potion_count': 2, 'ploc': [(0, 0)], (0, 2): 'M', (2, 0): 'M', (0, 1): 'D', (1, 2): 'O'}


In [9]:
import copy as cp
c=read_initial_state("Tests/test1.in")
d=cp.deepcopy(c)
d=move(cp.deepcopy(d),0,(0,1))
d=move(cp.deepcopy(d),0,(0,1))
d=move(cp.deepcopy(d),0,(0,1))
d=move(cp.deepcopy(d),0,(1,0))
print(d)

{'row': 4, 'col': 4, 'potion_count': 1, 'ploc': [(1, 3), (0, 0)], (2, 0): 'M', (1, 2): 'O'}


# BFS

Explainations TBA

In [ ]:
def BFS(file_address):
    i=0;
    frontier_queue=[]
    visited_states=[]
    order={}
    frontier_queue.append(read_initial_state(file_address))
    frontier_queue[0]["order"]=""
    while(frontier_queue):
        if(i==3):
            break
        i++
        current_state=frontier_queue.pop()
        print(frontier_queue)
        if (current_state in visited_states):
            continue
        visited_states.append(current_state)
        for player in range(len(current_state["ploc"])):
            for action in ACTIONS:
                new_state=move(current_state,player,action)
                if(new_state==0 or (new_state in visited_states)):
                    continue
                new_state["order"]=current_state["order"]+" "+str(player)+"("+str(action[0])+" "+str(action[1])+")"
                if(is_goal_state(new_state)):
                    return new_state["order"]
                frontier_queue.append(new_state) 
    return 0

In [ ]:
print(BFS("Tests/test1.in"))